# Step (n-2): Generate Feature Vector

In [1]:
%pylab inline
import pandas as pd
import contributions
import votes
import bills
import sqlCommands
from peoplefinder import PeopleFinder
from operator import attrgetter
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
from tqdm import tqdm

def subject_match(engine,A):
    query = ("SELECT bill_type,bill_number,status,result FROM allbills3 WHERE (subjects LIKE '%%{" + A
                     + ",%%' OR subjects LIKE '%%,"+A+",%%' OR subjects LIKE '%%,"+A+"}%%');")
    #use query and put in DataFrame
    foo = pd.read_sql_query(query,engine)
    return foo

def get_subject_features(engine):
    query = "SELECT * FROM subs_morebills_hair"
    foo = pd.read_sql_query(query, engine)
    return foo

def clean_subject(a):
    try:
        quoteloc = a.index("'") #may get issues with multiple single quotes in one topic.
        a = a[:quoteloc]+"'"+a[quoteloc:]
    except:
        pass #no quotes found
    #need to add " if there are any spaces in the subject
    try:
        quoteloc = a.index(" ") #will go to except if there are no spaces in the subject
        a = '\"'+a+'\"'
    except:
        pass #no spaces found
    return a

def remove_paren(a):
    try:
        parenloc = a.index("(")
        a = a[:parenloc]+a[parenloc+1:]
    except:
        pass
    try:
        parenloc = a.index(")")
        a = a[:parenloc]+a[parenloc+1:]
    except:
        pass
    return a

Populating the interactive namespace from numpy and matplotlib


In [2]:
#set up postgresql engine
dbname = 'legislatr'
engine = sqlCommands.get_engine(dbname)
conn = psycopg2.connect("dbname='legislatr' user='lordluen' host='localhost'")
cur = conn.cursor()

In [3]:
#load in subject features
sub_feat_temp = get_subject_features(engine)
sub_feat = pd.DataFrame()
sub_feat['subjects'] = sub_feat_temp.subjects
sub_feat['q_subjects'] = sub_feat['subjects'].map( lambda x: clean_subject(x)) #subjects for querying.
sub_feat['df_subjects'] = sub_feat['subjects'].map( lambda x:remove_paren(x))

In [4]:
print(sub_feat.subjects.size)
sub_feat.head(3)

406


,subjects,q_subjects,df_subjects
0,Access Board,"""Access Board""",Access Board
1,Administrative Conference of the U.S.,"""Administrative Conference of the U.S.""",Administrative Conference of the U.S.
2,Administrative law and regulatory procedures,"""Administrative law and regulatory procedures""",Administrative law and regulatory procedures


In [6]:
#create a dataframe will bill in first column, pass or fail in second column (will need to clean), and features in reamaining columns
#first get a list of bills.
#bills_subjects = pd.read_sql("SELECT subjects FROM allbills;",engine)
bills_subjects = pd.read_sql("SELECT subjects FROM allbills3;",engine)
#bills_features = pd.read_sql("SELECT bill_type,bill_number,status,result FROM allbills;",engine)
bills_features = pd.read_sql("SELECT bill_type,num_amends,bill_number,status,result,final_result FROM allbills3;",engine)
print(bills_features['bill_type'].size)
bills_features.head(3)
#bills_subjects.head(3)

38340


,bill_type,num_amends,bill_number,status,result,final_result
0,s,0,1870,REPORTED,2,0
1,s,0,125,REFERRED,2,0
2,s,0,1027,REFERRED,2,0


In [7]:
bills_subjects.head(3)

,subjects
0,"{""Administrative law and regulatory procedures..."
1,"{""Aquatic ecology"",""Congressional oversight"",F..."
2,"{""Congressional oversight"",""Department of Heal..."


In [7]:
#CELL NO LONGER IN USE
#second make new dataframe with just type, number, and result.
#bills_features = pd.DataFrame()
#bills_features[['type','number','result']] = bills[['type','number','result']]

In [8]:
#bills_features.head(3)

In [8]:
#query type and number for bills that match each subject, map into df.
#loop over subjects.
for i in tqdm(range(0,sub_feat['subjects'].size)):
#get subject.
#qsub = sub_feat['q_subjects'].iloc[i] #pull out subject to search for
#currently using a postgresql query to get bill info, might be faster to search the dataframe I already have loaded.
#matches = subject_matches(engine,qsub) #get bill identifiers that have those subjects
    qsub = sub_feat['subjects'].iloc[i] #pull out subject to search for
    csub = sub_feat['df_subjects'].iloc[i] #subject to be column name
    boolcount = bills_subjects.subjects.str.contains(qsub, na=False)
    bills_features[csub] = boolcount.map({False:0,True:1}).astype(int) #subjects for querying.

100%|██████████| 406/406 [00:12<00:00, 33.76it/s]


In [9]:
#check that it worked.
print(bills_features.loc[0][bills_features.loc[0] == 1])
print(bills_subjects['subjects'].iloc[0])

Administrative law and regulatory procedures    1
Child care and development                      1
Child safety and welfare                        1
Congress                                        1
Crime victims                                   1
Department of Health and Human Services         1
Executive agency funding and structure          1
Family services                                 1
Homelessness and emergency shelter              1
Housing and community development funding       1
Human trafficking                               1
Indian social and development programs          1
International law and treaties                  1
Sex offenses                                    1
Visas and passports                             1
Name: 0, dtype: object
{"Administrative law and regulatory procedures","Adoption and foster care","Advisory bodies","Bank accounts, deposits, capital","Child care and development","Child health","Child safety and welfare","Congressional oversight","C

In [12]:
print(np.max(bills_features['num_amends']))
bills_features.head(3)

621


,bill_type,num_amends,bill_number,status,result,final_result,Access Board,Administrative Conference of the U.S.,Administrative law and regulatory procedures,Afghanistan,...,Water use and supply,Watersheds,West Bank,Wetlands,Women's health,Women's rights,World health,Wyoming,Yemen,Zimbabwe
0,s,0,1870,REPORTED,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,s,0,125,REFERRED,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,s,0,1027,REFERRED,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#check number of success vs fail
print(bills_features[bills_features['final_result'] == 0]['bill_number'].size)
print(bills_features[bills_features['final_result'] == 1]['bill_number'].size)
print(bills_features[bills_features['final_result'] == 2]['bill_number'].size)
print(np.unique(bills_features[bills_features['final_result'] == 2]['congress']))

29054
1104
8182


KeyError: 'congress'

In [15]:
#save results to database.
sqlCommands.write_to_database(dbname,'features',bills_features,engine) #this is large, give it a few minutes

# STOP HERE!

In [46]:
print(np.sum(bills_features.outcome))
print(bills_features.outcome.size)

1054
1080


It appears the category passage is biased towards passing bills......
May need to redo everything using different categories.

In [47]:
#playing with other categories ['amendment' 'recommit' 'passage-suspension' 'procedural' 'passage' 'cloture']
bills = pd.read_sql("SELECT * FROM bills WHERE category LIKE 'recommit';",engine)
print(bills['index'].size)
bills.head(3)

314


,index,category,chamber,congress,date,number,result,subjects,top_subject,type
0,55,recommit,h,113,2013-10-30 14:19:00,992,Failed,"{""Banking and financial institutions regulatio...",Finance and financial sector,hr
1,15,recommit,h,113,2013-02-06 11:52:00,444,Failed,"{""Budget deficits and national debt"",""Budget p...",Economics and public finance,hr
2,21,recommit,h,113,2013-09-19 17:59:00,3102,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr


In [48]:
bills = pd.read_sql("SELECT * FROM bills;",engine)
print(bills['index'].size)
bills.head(3)

5625


,index,category,chamber,congress,date,number,result,subjects,top_subject,type
0,0,amendment,h,113,2013-06-20 11:09:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
1,1,amendment,h,113,2013-11-21 11:22:00,1900,Failed,"{""Administrative law and regulatory procedures...",Energy,hr
2,2,amendment,h,113,2013-07-10 14:32:00,2609,Failed,"{""Administrative law and regulatory procedures...",Economics and public finance,hr


In [54]:
test = pd.unique(bills.number)
print(test)

[1947 1900 2609 ...,  735  764 4890]


In [58]:
#for i in range(0,test.size):
q = 'number == '+str(test[0])
foo = bills.query(q)

In [59]:
foo.head(5)

,index,category,chamber,congress,date,number,result,subjects,top_subject,type
0,0,amendment,h,113,2013-06-20 11:09:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
27,23,amendment,h,113,2013-06-20 11:18:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
56,50,amendment,h,113,2013-06-20 13:03:00,1947,Agreed to,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
98,93,amendment,h,113,2013-06-20 10:50:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr
102,97,amendment,h,113,2013-06-19 18:36:00,1947,Failed,"{""Administrative law and regulatory procedures...",Agriculture and food,hr


In [ ]:
#need to find amendment number and replace bill info for amendment votes with info from the amendments.